In [1]:
%reload_ext autoreload
%autoreload 2

import asyncio
import nest_asyncio
import os
import sys
from typing import ClassVar, Optional, Any, Dict, List, Union

import anthropic
from dotenv import load_dotenv
import openai
import pandas as pd
import pydantic

sys.path.append('../')
from lattereview.providers.openai_provider import OpenAIProvider
from lattereview.agents.scoring_reviewer import ScoringReviewer

# Load environment variables from .env file
load_dotenv('../.env')
print(os.getenv('OPENAI_API_KEY'))

# Enable asyncio in Jupyter
nest_asyncio.apply()

Failed to update token costs. Using static costs.


sk-cq_M0pNgHhCFnlDOCMnagYA1l2X7Yea5CL0ci5pZMNT3BlbkFJ0m0x9wm5M_EstX5SjLu_kdwGMDYjkUdviNPs4pe9cA


/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/tokencost/constants.py:69: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  logger.error("Failed to update token costs. Using static costs.")


In [2]:
df = pd.read_excel('data.xlsx')
df.head()

,PMID,PMC ID,Title,Author(s),Author(s) Full Name,Author(s) Affiliation,Corporate Author,Collaborator(s),Collaborator(s) Full Name,Collaborator(s) Affiliation,...,Manuscript Identifier,Secondary Source ID,Publishing Model,Comment on,Comment in,Erratum in,Erratum for,Corrected and Republished in,Corrected and Republished from,Owner
0,39618885,PMC11608428,Learning to Generate Context-Sensitive Backcha...,Bilalpur M||Inan M||Zeinali D||Cohn JF||Alikha...,"Bilalpur, Maneesh||Inan, Mert||Zeinali, Dorsa|...","University of Pittsburgh, Pittsburgh, Pennsylv...",NaN,NaN,NaN,NaN,...,NIHMS2038099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
1,39596657,PMC11593565,Assessing the Impact and Cost-Effectiveness of...,Liu SH||Weber ES||Manz KE||McCarthy KJ||Chen Y...,"Liu, Shelley H||Weber, Ellerie S||Manz, Kather...",Department of Population Health Science and Po...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
2,39594314,PMC11591420,Bridging Employees' Perceptions of Corporate S...,Wang Z||Carroll S||Wang EH,"Wang, Zheni||Carroll, Steve||Wang, Eric H",Department of Management/International-Busines...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
3,39531013,NaN,The influence of active agent motility on SIRS...,Kailasham R||Khair AS,"Kailasham, R||Khair, Aditya S","Department of Chemical Engineering, Carnegie M...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
4,39527845,NaN,Coupled infectious disease and behavior dynami...,Reitenbach A||Sartori F||Banisch S||Golovin A|...,"Reitenbach, Andreas||Sartori, Fabio||Banisch, ...","Sociology and Computational Social Science, Ka...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM


In [3]:
print(os.getenv("OPENAI_API_KEY", ""))
openanai_provider = OpenAIProvider(model="gpt-4o-mini")
question = "What is the capital of France?"
asyncio.run(openanai_provider.get_response(question, temperature=0.9))

sk-cq_M0pNgHhCFnlDOCMnagYA1l2X7Yea5CL0ci5pZMNT3BlbkFJ0m0x9wm5M_EstX5SjLu_kdwGMDYjkUdviNPs4pe9cA


/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


('The capital of France is Paris.',
 {'input_cost': 1.05e-06, 'output_cost': 4.2e-06, 'total_cost': 5.25e-06})

In [4]:
agent = ScoringReviewer(
    provider=OpenAIProvider(),
    name="Pouria",
    backstory="an expert reviewer and researcher!",
    input_description = "article title",
    temperature=0.1,
    reasoning = "brief",
    max_tokens=100,
    review_criteria="Look for articles that certainly do not employ any AI or machine learning agents",
    scoring_set=[1, 2],
    support_agent = "test",
    scoring_rules='Your scores should be either 1 or 2. 1 means that the paper does not meet the criteria, and 2 means that the paper meets the criteria.',
    # examples = [
    #     {
    #         "input": "bridging employees' perceptions of corporate social responsibility, sense-making for meaningfulness, and work engagement for successful self-regulation.",
    #         "SCORE": 2
    #     }
#    ]
)

# Dummy input
text_list = df.Title.str.lower().tolist()
print("Inputs:\n\n", '\n'.join(text_list[:3]))

# Dummy review
print("\nOutputs:\n\n", '\n'.join(asyncio.run(agent.review_items(text_list[:3]))))

# Dummy costs
print("\nCosts:\n")
for item in agent.memory:
    print(item['cost'])


Inputs:

 learning to generate context-sensitive backchannel smiles for embodied ai agents with applications in mental health dialogues.
assessing the impact and cost-effectiveness of exposome interventions on alzheimer's disease: a review of agent-based modeling and other data science methods for causal inference.
bridging employees' perceptions of corporate social responsibility, sense-making for meaningfulness, and work engagement for successful self-regulation.

Outputs:

 {"score":1,"reasoning":"The article explicitly discusses generating context-sensitive backchannel smiles for embodied AI agents, indicating a clear use of AI technology."}
{"score":1,"reasoning":"The article discusses agent-based modeling and data science methods, which typically involve AI or machine learning techniques, indicating it does not meet the criteria of not employing such methods."}
{"score":2,"reasoning":"The article focuses on employee perceptions and engagement without any indication of employing A

In [5]:
agent.identity

{'system_prompt': "Your name is <<Pouria>> and you are <<an expert reviewer and researcher!>>. Your task is to review input itmes with the following description: <<article title>>. Your final output should have the following keys: score (<class 'int'>), reasoning (<class 'str'>).",
 'item_prompt': 'Review the item below and evaluate it against the following criteria: Input item: <<${item}$>> Review criteria: <<Look for articles that certainly do not employ any AI or machine learning agents>> Now score the article based on your evaluation and by choosing one of the following values: [1, 2]. If you are highly in doubt about what to score, score "0". Your scoring should be based on the following rules: <<Your scores should be either 1 or 2. 1 means that the paper does not meet the criteria, and 2 means that the paper meets the criteria.>> You must also provide a brief (1 sentence) reasoning for your scoring. First reason then score!',
 'temperature': 0.1,
 'max_tokens': 100}

In [6]:
agent.memory

[{'identity': {'system_prompt': "Your name is <<Pouria>> and you are <<an expert reviewer and researcher!>>. Your task is to review input itmes with the following description: <<article title>>. Your final output should have the following keys: score (<class 'int'>), reasoning (<class 'str'>).",
   'item_prompt': 'Review the item below and evaluate it against the following criteria: Input item: <<${item}$>> Review criteria: <<Look for articles that certainly do not employ any AI or machine learning agents>> Now score the article based on your evaluation and by choosing one of the following values: [1, 2]. If you are highly in doubt about what to score, score "0". Your scoring should be based on the following rules: <<Your scores should be either 1 or 2. 1 means that the paper does not meet the criteria, and 2 means that the paper meets the criteria.>> You must also provide a brief (1 sentence) reasoning for your scoring. First reason then score!',
   'temperature': 0.1,
   'max_tokens'

In [7]:
agent.cost_so_far

0.00013815000000000002